**Requirements:**
- scikit-learn
- numpy
- joblib
- Your own `utils/metrics.py` and `utils/visualization.py` scripts

In [ ]:
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from utils.metrics import calculate_metrics, print_metrics_summary
from utils.visualization import save_visualizations

In [ ]:
def run_random_forest(train_X, valid_X, train_y, valid_y, test_X, test_y, 
                      model_name="RandomForest", class_names=None, 
                      save_dir="saved_models"):
    os.makedirs(save_dir, exist_ok=True)
    print("\n=== RANDOM FOREST CLASSIFICATION ===")

    tuned_parameters = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5],
        'class_weight': ['balanced', None]
    }

    X = np.concatenate((train_X, valid_X), axis=0)
    y = np.concatenate((train_y, valid_y), axis=0)
    test_fold = np.concatenate([
        np.full(train_y.shape[0], -1),
        np.zeros(valid_y.shape[0])
    ])

    clf = GridSearchCV(
        estimator=RandomForestClassifier(random_state=42),
        param_grid=tuned_parameters,
        cv=PredefinedSplit(test_fold),
        n_jobs=-1,
        verbose=2,
        scoring='f1_weighted'
    )
    clf.fit(X, y)

    best_model = clf.best_estimator_

    test_X_flat = test_X.reshape(test_X.shape[0], -1) if test_X.ndim > 2 else test_X
    test_pred = best_model.predict(test_X_flat)
    metrics = calculate_metrics(test_y, test_pred, model_name)

    print("\n=== BEST PARAMETERS ===")
    print(clf.best_params_)
    print_metrics_summary(metrics)

    save_visualizations(
        model=best_model,
        x_data=test_X_flat,
        y_true=test_y,
        y_pred=test_pred,
        model_name=model_name,
        class_names=class_names
    )

    model_path = os.path.join(save_dir, f"{model_name}.joblib")
    joblib.dump(best_model, model_path)
    print(f"\nModel saved to {model_path}")

    metrics_path = os.path.join(save_dir, f"{model_name}_metrics.txt")
    with open(metrics_path, 'w') as f:
        f.write(f"Best Parameters:\n{clf.best_params_}\n\n")
        f.write("Evaluation Metrics:\n")
        for k, v in metrics.items():
            f.write(f"{k}: {v}\n")
    print(f"Metrics saved to {metrics_path}")

    return best_model, metrics

In [ ]:
def load_random_forest(model_path):
    """Load a saved RandomForest model"""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"No model found at {model_path}")
    return joblib.load(model_path)

##  Example Usage (Assumes Data Is Loaded)

*Uncomment and adapt the following code for real use:*

In [ ]:
# best_model, metrics = run_random_forest(
#     train_X, valid_X, train_y, valid_y,
#     test_X, test_y,
#     model_name="MyRandomForestModel",
#     class_names=["Class1", "Class2", "Class3"]
# )